In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [3]:
import os

In [4]:
print(os.getcwd())

/content


In [5]:
%cd /content/gdrive/MyDrive/Masters/Year_1/Semester_2/CS_505/Project/NN

/content/gdrive/MyDrive/Masters/Year_1/Semester_2/CS_505/Project/NN


In [6]:
from models.model_baselines import BaselineClassifier

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from typing import List, Dict, Optional, Tuple, Any
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, BatchNormalization
import tensorflow as tf
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold

In [8]:
print(tf.test.gpu_device_name())

/device:GPU:0


In [9]:
%ls

'Copy of MT5_final_embeddings.ipynb'   mlp_tfidf_cv_2
'MD5_1 (1).ipynb'                      models/
 MD5_1_English.ipynb                   MT5_final_embeddings.ipynb
 mlp_tfidf_cv                          Untitled


In [10]:
train_df = pd.read_csv('../Data/Train.csv')

In [11]:
le = LabelEncoder()
train_df.Label = le.fit_transform(train_df.Label)

In [12]:
Y_train = np_utils.to_categorical(train_df['Label'], np.unique(train_df['Label']).shape[0])

In [13]:
def get_model(size):
  print('Building model...')
  model = Sequential()
  model.add(Dense(2048, input_shape=(size,)))
  model.add(Activation('relu'))
  model.add(Dropout(0.2))
  model.add(Dense(20))
  model.add(Activation('softmax'))

  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model
  #make one, takes in, double that amount, english and chichewa

In [14]:
def get_results(model,epochs,train_dataset,test_dataset,y_test):
  model.fit(train_dataset,
            epochs=epochs, batch_size=32,
            verbose=1)
  print(model.evaluate(test_dataset))
  ans = model.predict(test_dataset)
  print(accuracy_score(y_test, np_utils.to_categorical(np.argmax(ans,axis=1),num_classes=20)))
  print(classification_report(y_test, np_utils.to_categorical(np.argmax(ans,axis=1),num_classes=20)))

In [15]:
print(Y_train.shape)
print(train_df.shape)

(1436, 20)
(1436, 3)


In [16]:
kf = KFold(n_splits=3, random_state=42, shuffle=True)

In [19]:
for train_index, test_index in kf.split(train_df.Text):
  X_train, X_test = train_df.iloc[train_index].Text, train_df.iloc[test_index].Text
  #y_train, y_test = train_df.iloc[train_index].Label, train_df.iloc[test_index].Label
  y_train, y_test = Y_train[train_index], Y_train[test_index]
  #print(X_train.shape)
  #print(y_train.shape)
  #print(X_test.shape)
  #print(y_test.shape)
  #continue
  #Y_train
  tfidf = TfidfVectorizer()
  tfidf_train = tfidf.fit_transform(X_train).toarray()
  tfidf_test = tfidf.transform(X_test).toarray()
  train_dataset = tf.data.Dataset.from_tensor_slices((tfidf_train, y_train))

  test_dataset = tf.data.Dataset.from_tensor_slices((tfidf_test, y_test))

  BATCH_SIZE = 32
  SHUFFLE_BUFFER_SIZE = 100

  train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
  test_dataset = test_dataset.batch(BATCH_SIZE)
  get_results(get_model(tfidf_train.shape[1]),50,train_dataset,test_dataset,y_test)

(957,)
(957, 20)
(479,)
(479, 20)
(957,)
(957, 20)
(479,)
(479, 20)
(958,)
(958, 20)
(478,)
(478, 20)


In [ ]:
#f1 macro 47 43 47
#accuracy 64 60 64

In [21]:
print((47+43+47)/3)
print((64+60+64)/3)

45.666666666666664
62.666666666666664


# Count Vectorizer

In [ ]:
cv = CountVectorizer()
cv_train = cv.fit_transform(X_train).toarray()
cv_test = cv.transform(X_test).toarray()

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((cv_train, y_train))

test_dataset = tf.data.Dataset.from_tensor_slices((cv_test, y_test))

BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [ ]:
get_results(get_model(cv_train.shape[1]),10,train_dataset,test_dataset,y_test)

Building model...
Epoch 1/10
32/32 [==============================] - 1s 29ms/step - loss: 1.8525 - accuracy: 0.5244
Epoch 2/10
32/32 [==============================] - 1s 28ms/step - loss: 0.0762 - accuracy: 0.9900
Epoch 3/10
32/32 [==============================] - 1s 28ms/step - loss: 0.0248 - accuracy: 0.9960
Epoch 4/10
32/32 [==============================] - 1s 28ms/step - loss: 0.0109 - accuracy: 0.9970
Epoch 5/10
32/32 [==============================] - 1s 28ms/step - loss: 0.0140 - accuracy: 0.9980
Epoch 6/10
32/32 [==============================] - 1s 28ms/step - loss: 0.0113 - accuracy: 0.9960
Epoch 7/10
32/32 [==============================] - 1s 32ms/step - loss: 0.0066 - accuracy: 0.9970
Epoch 8/10
32/32 [==============================] - 1s 31ms/step - loss: 0.0076 - accuracy: 0.9980
Epoch 9/10
32/32 [==============================] - 1s 32ms/step - loss: 0.0080 - accuracy: 0.9980
Epoch 10/10
14/14 [==============================] - 1s 13ms/step - loss: 1.5019 - accuracy

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
get_results(get_model(cv_train.shape[1]),25,train_dataset,test_dataset,y_test)

Building model...
Epoch 1/25
32/32 [==============================] - 1s 29ms/step - loss: 1.8503 - accuracy: 0.5184
Epoch 2/25
32/32 [==============================] - 1s 27ms/step - loss: 0.0808 - accuracy: 0.9861
Epoch 3/25
32/32 [==============================] - 1s 27ms/step - loss: 0.0237 - accuracy: 0.9960
Epoch 4/25
32/32 [==============================] - 1s 27ms/step - loss: 0.0146 - accuracy: 0.9960
Epoch 5/25
32/32 [==============================] - 1s 27ms/step - loss: 0.0151 - accuracy: 0.9960
Epoch 6/25
32/32 [==============================] - 1s 28ms/step - loss: 0.0131 - accuracy: 0.9960
Epoch 7/25
32/32 [==============================] - 1s 28ms/step - loss: 0.0104 - accuracy: 0.9960
Epoch 8/25
32/32 [==============================] - 1s 28ms/step - loss: 0.0102 - accuracy: 0.9970
Epoch 9/25
32/32 [==============================] - 1s 28ms/step - loss: 0.0069 - accuracy: 0.9970
Epoch 10/25
32/32 [==============================] - 1s 29ms/step - loss: 0.0153 - accuracy

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
get_results(get_model(cv_train.shape[1]),50,train_dataset,test_dataset,y_test)

Building model...
Epoch 1/50
32/32 [==============================] - 1s 28ms/step - loss: 1.9028 - accuracy: 0.5114
Epoch 2/50
32/32 [==============================] - 1s 28ms/step - loss: 0.0888 - accuracy: 0.9881
Epoch 3/50
32/32 [==============================] - 1s 28ms/step - loss: 0.0283 - accuracy: 0.9950
Epoch 4/50
32/32 [==============================] - 1s 28ms/step - loss: 0.0156 - accuracy: 0.9970
Epoch 5/50
32/32 [==============================] - 1s 28ms/step - loss: 0.0127 - accuracy: 0.9960
Epoch 6/50
32/32 [==============================] - 1s 29ms/step - loss: 0.0102 - accuracy: 0.9960
Epoch 7/50
32/32 [==============================] - 1s 28ms/step - loss: 0.0073 - accuracy: 0.9970
Epoch 8/50
32/32 [==============================] - 1s 28ms/step - loss: 0.0095 - accuracy: 0.9960
Epoch 9/50
32/32 [==============================] - 1s 28ms/step - loss: 0.0082 - accuracy: 0.9970
Epoch 10/50
32/32 [==============================] - 1s 28ms/step - loss: 0.0083 - accuracy

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
get_results(get_model(cv_train.shape[1]),100,train_dataset,test_dataset,y_test)

Building model...
Epoch 1/100
32/32 [==============================] - 1s 29ms/step - loss: 1.8681 - accuracy: 0.5134
Epoch 2/100
32/32 [==============================] - 1s 28ms/step - loss: 0.0656 - accuracy: 0.9930
Epoch 3/100
32/32 [==============================] - 1s 28ms/step - loss: 0.0229 - accuracy: 0.9960
Epoch 4/100
32/32 [==============================] - 1s 28ms/step - loss: 0.0124 - accuracy: 0.9960
Epoch 5/100
32/32 [==============================] - 1s 28ms/step - loss: 0.0121 - accuracy: 0.9960
Epoch 6/100
32/32 [==============================] - 1s 28ms/step - loss: 0.0131 - accuracy: 0.9960
Epoch 7/100
32/32 [==============================] - 1s 28ms/step - loss: 0.0141 - accuracy: 0.9960
Epoch 8/100
32/32 [==============================] - 1s 28ms/step - loss: 0.0095 - accuracy: 0.9960
Epoch 9/100
32/32 [==============================] - 1s 28ms/step - loss: 0.0099 - accuracy: 0.9960
Epoch 10/100
32/32 [==============================] - 1s 28ms/step - loss: 0.0143 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
